In [ ]:
from django.conf import settings
print(settings.DEBUG)
assert settings.DEBUG, 'DO NOT RUN IN PRODUCTION'
mindate = '2017-06-01'
Story.objects.filter(created__date__lte=mindate).delete()
ImageFile.objects.photos().filter(**kwargs).delete()
Issue.objects.filter(publication_date__lte=mindate).delete()
Contributor.objects.filter(byline=None).delete()
ImageFile.objects.filter(contributor=None, storyimage=None, frontpagestory=None).delete()

In [ ]:
print(
    Issue.objects.count(),
    ImageFile.objects.count(),
    Contributor.objects.count(),
)

In [70]:
from tqdm import tqdm_notebook, tqdm

import requests
import os
from django.core.files.base import ContentFile

def save_request_data(file_field, response):
    filename = os.path.basename(file_field.name)
    file_field.save(filename, response.raw, save=True)
    
def get_file_url(endpoint, pk, field, apibase='http://universitas.no/api/'):
    apiurl = os.path.join(*map(str, (apibase, endpoint, pk, '')))
    return requests.get(apiurl).json().get(field)

def get_pdf_from_api(pi):
    try: 
        file = pi.pdf.file
    except FileNotFoundError:
        print(f'missing: {pi.pdf}')
    else:
        print(f'exists: {pi.pdf}')
        return
    pdfurl = get_file_url('pdfs', pi.pk, 'pdf')
    response = requests.get(pdfurl, stream=True)
    save_request_data(pi.pdf, response)
    print(pi.get_cover_page())
    
def get_imagefile_from_api(img):
    try: 
        file = img.source_file.file
    except FileNotFoundError:
        url = get_file_url('images', img.pk, 'large')
        response = requests.get(url, stream=True)
        save_request_data(img.source_file, response)
    return img.source_file.name


In [71]:
imagefiles = tqdm(ImageFile.objects.all(), mininterval=0.002)
for im in imagefiles:
    fn = get_imagefile_from_api(im)
    imagefiles.set_description(f'{fn[:60]:<60}')
    

2017/6/13-NYH-LeneAntonsen-HH-6.jpg                         : 100%|██████████| 344/344 [00:00<00:00, 350.05it/s] 
